# Tutorial MongoDB CRUD and Aggregation, <div style="color:red">Module Assigment 1</div>

----
student's name:

student's email:

----

<br><br>


<div style="color:red">Attention, this tutorial is an Module Assigment, it is used for grade. I refere to the module course where you should hand in this notebook with your code, description and explanations</div>


<br><br>



Download the mongodb server and install it on your machine.
the notebook requires `pymongo`: `pip install pymongo`  this as well.


Mac command for running the mongoDB Server after installing the server using brew: `brew services start mongodb/brew/mongodb-community`


Make yourself familiar with the possibilities to access and wrangle information using pymongo and the underlying query mechanism of mongoDB. A document describing the aggregation pipeline of mongoDB is located in the moodle course.




#### Connection establishment

Python based Connection Management, URL of server is necessary, typically `localhost`

Database name: _`shopDB`_

Collections are referenced using the name of the collections.

Collection is created if not already existant in DB


In [ ]:
from pymongo import MongoClient
from bson import ObjectId
from datetime import datetime

client = MongoClient("mongodb://localhost:27017")
db = client["shopDB"]

users = db["users"]
products = db["products"]
orders = db["orders"]
reviews = db["reviews"]


# Reset
users.delete_many({})
products.delete_many({})
orders.delete_many({})
reviews.delete_many({})



#### Database content 
some entries are added (create / inserted) to the corresponding collection

In [ ]:

# Insert users
users.insert_many([
    {
        "_id": ObjectId("65f100000000000000000001"),
        "name": "Alice Müller",
        "email": "alice@example.com",
        "createdAt": datetime(2025, 1, 10, 9, 0, 0),
        "address": {"street": "Main Street 1", "city": "Berlin", "zip": "10115", "country": "DE"},
        "tags": ["premium", "newsletter"],
    },
    {
        "_id": ObjectId("65f100000000000000000002"),
        "name": "Bob Schmidt",
        "email": "bob@example.com",
        "createdAt": datetime(2025, 1, 12, 15, 30, 0),
        "address": {"street": "Oak Avenue 5", "city": "Hamburg", "zip": "20095", "country": "DE"},
        "tags": ["standard"],
    },
])

# Insert products
products.insert_many([
    {"_id": ObjectId("65f200000000000000000001"), "name": "Mechanical Keyboard",
     "category": "electronics", "price": 129.99, "stock": 42,
     "tags": ["keyboard", "mechanical", "gaming"], "createdAt": datetime(2025,1,8,10)},
    {"_id": ObjectId("65f200000000000000000002"), "name": "Noise-Cancelling Headphones",
     "category": "electronics", "price": 199.00, "stock": 15,
     "tags": ["audio","headphones"], "createdAt": datetime(2025,1,9,11,30)},
    {"_id": ObjectId("65f200000000000000000003"), "name": "Ergonomic Office Chair",
     "category": "furniture", "price": 299.00, "stock": 7,
     "tags": ["office","chair"], "createdAt": datetime(2025,1,11,13)},
])

# Insert orders
orders.insert_many([
    {
        "_id": ObjectId("65f300000000000000000001"),
        "userId": ObjectId("65f100000000000000000001"),
        "createdAt": datetime(2025,1,15,10,15),
        "status": "shipped",
        "items": [
            {"productId": ObjectId("65f200000000000000000001"), "nameSnapshot": "Mechanical Keyboard", "quantity": 1, "priceAtPurchase": 119.99},
            {"productId": ObjectId("65f200000000000000000002"), "nameSnapshot": "Noise-Cancelling Headphones", "quantity": 1, "priceAtPurchase": 189.00},
        ],
        "shippingAddress": {"street":"Main Street 1","city":"Berlin","zip":"10115","country":"DE"},
        "payment": {"method":"credit_card","paid":True,"totalAmount":308.99},
    },
    {
        "_id": ObjectId("65f300000000000000000002"),
        "userId": ObjectId("65f100000000000000000002"),
        "createdAt": datetime(2025,1,16,16,45),
        "status": "processing",
        "items": [
            {"productId": ObjectId("65f200000000000000000003"), "nameSnapshot": "Ergonomic Office Chair", "quantity": 1, 
             "priceAtPurchase": 289.00},
        ],
        "shippingAddress": {"street":"Oak Avenue 5","city":"Hamburg","zip":"20095","country":"DE"},
        "payment": {"method":"paypal","paid":False,"totalAmount":289.00},
    },
])

# Insert reviews
reviews.insert_many([
    {"_id": ObjectId("65f400000000000000000001"),
     "productId": ObjectId("65f200000000000000000001"),
     "userId": ObjectId("65f100000000000000000001"),
     "rating": 5, "title": "Great keyboard!",
     "text": "Feels amazing to type, perfect for coding.",
     "createdAt": datetime(2025,1,17,9)},
    {"_id": ObjectId("65f400000000000000000002"),
     "productId": ObjectId("65f200000000000000000002"),
     "userId": ObjectId("65f100000000000000000002"),
     "rating": 4, "title": "Good sound",
     "text": "Noise cancelling is solid, but a bit tight.",
     "createdAt": datetime(2025,1,18,14,30)},
]

)

print("Data inserted successfully")

Adding individual Entries are also possible.

In [ ]:



# 3.1 CREATE: a User
new_user = {
    "name": "Charlie Weber",
    "email": "charlie@example.com",
    "createdAt": datetime.utcnow(),
    "address": {
        "street": "Example St 10",
        "city": "Munich",
        "zip": "80331",
        "country": "DE",
    },
    "tags": ["standard"],
}
insert_result = users.insert_one(new_user)
print("Inserted new user with _id:", insert_result.inserted_id)



In [ ]:

# 3.2 READ: products of categoryelectronics, price < 150
print("\nElectronics products with price < 150:")
for doc in products.find({"category": "electronics", "price": {"$lt": 150}}):
    print(doc)
'
# READ: Alice's Orders  
alice = users.find_one({"email": "alice@example.com"})
print("\nOrders for Alice:")
for order in orders.find({"userId": alice["_id"]}):
    print(order)

# READ: get reviews for specific product
keyboard = products.find_one({"name": "Mechanical Keyboard"})
print("\nReviews for Mechanical Keyboard:")
for review in reviews.find({"productId": keyboard["_id"]}):
    print(review)

# 3.3 UPDATE: change order status
print("\nUpdating order status for Bob's order...")
orders.update_one(
    {"_id": ObjectId("65f300000000000000000002")},
    {"$set": {"status": "shipped", "payment.paid": True}},
)
print("Updated order:\n", orders.find_one({"_id": ObjectId("65f300000000000000000002")}))

# UPDATE: Increase all elements by 10 money ;)
print("\nIncreasing price of all electronics products by 10...")
products.update_many(
    {"category": "electronics"},
    {"$inc": {"price": 10}},
)
for doc in products.find({"category": "electronics"}):
    print(doc)

# 3.4 DELETE
print("\nDeleting reviews with rating <= 2 (if any)...")
delete_result = reviews.delete_many({"rating": {"$lte": 2}})
print("Deleted count:", delete_result.deleted_count)


# Task (MongoDB,  Nested Documents)

<div style="color:orange">Create for each task a specific python function (<code>taks1(...), task2(...), task3(...)</code>)!</div> 

<br>

<div style="color:red">Make also sure to compare your solution against the solution provided by chatgpt. Discuss overlaps or divergences of the solutions.</div>

In addition to the implementation tasks, discuss where atomicity and multi-document transactions are required in this scenario (user creation, stock validation and update, order insertion).
Briefly explain the potential consequences if no atomicity / transaction support is used (e.g. race conditions, overselling, inconsistent orders).
Finally, provide a concrete solution that uses MongoDB transactions (via pymongo and sessions) to ensure that user handling, stock updates, and order creation are executed atomically.

<br>

**Be aware to combine all subTask Functions in the last function**

Implement python code to create a **new order in MongoDB** with the following requirements:


1. **User handling (embedded user document in the order)**  <code>taks1(...)</code>
   - The order is placed by a user identified by some unique field (e.g. `email`).  
   - Check in the MongoDB `users` collection whether this user already exists.  
   - If the user does **not** exist, insert a **new user document** into the `users` collection. Align the format of the new document with the previously stored documents.
   - In the order document, embed a **nested `user` subdocument** (e.g. containing `userId`, `name`, `email`, etc.), so that user information is directly available inside the order.


In [ ]:
def task1():
    pass


2. **Order content with embedded product descriptions**  <code>taks2(...)</code>
   - The order must contain **at least one item**.
   - Each item refers to a product stored in the `products` collection (e.g. via `productId`).  
   - For each item, fetch the corresponding product from `products` and embed a **nested product description** inside the order, for example:
     ```json
     {
       "productId": "...",
       "name": "...",
       "priceAtPurchase": ...,
       "quantity": ...
     }
     ```
   - This way, the order stores a **snapshot of the product data** at the time of purchase.


In [ ]:
def task2():
    pass


3. **Stock validation and update in MongoDB**  
   - Before inserting the order into the `orders` collection:
     - For each item, query the `products` collection and check whether there is **sufficient `stock`** available.  
     - If **any** product does not have enough stock, **abort** the operation (do not insert the order, do not modify stock).  
     - If all products have sufficient stock, **decrease the stock** in `products` for each item (e.g. using `$inc: { stock: -quantity }`).




In [ ]:
def task3():
    pass

4. **Order creation in the `orders` collection**  
   - Only after:
     - (a) the user has been ensured to exist (and created if necessary), and  
     - (b) all stock levels have been validated and updated,  
   - insert a new document into the `orders` collection.  
   - The order document should contain:
     - an `_id` or `orderId`,  
     - a `createdAt` timestamp,  
     - the embedded `user` subdocument,  
     - an `items` array containing the embedded product descriptions with quantities,  
     - optional payment/total fields (e.g. `totalAmount`).


In [1]:
def task4():
    pass

In [ ]:
def combinationOfAllTasks():
    pass

## Aggregations in MongoDB

We now want to apply specific aggregation operations to retrieve combined information from useser and their orders. The json file contains a larger dataset which has to be analyzed.

In [ ]:
import json
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["shopOrders"]

with open("orders.json", encoding="utf-8") as f:
    data = json.load(f)

db.users.insert_many(data["users"])
db.products.insert_many(data["products"])
db.orders.insert_many(data["orders"])
db.reviews.insert_many(data["reviews"])

Example pipeline for getting mean information for order values:

In [ ]:
pipeline = [
    {
        "$group": {
            "_id": None,
            "avgOrderValue": {"$avg": "$payment.totalAmount"},
            "minOrderValue": {"$min": "$payment.totalAmount"},
            "maxOrderValue": {"$max": "$payment.totalAmount"},
            "countOrders": {"$sum": 1},
        }
    }
]

list(db.orders.aggregate(pipeline))

### Get all Users, who ordered more than twice of the avarage price of an order

In [ ]:
pipeline = [
    # 1: Durchschnitt berechnen
    {
        "$group": {
            "_id": None,
            "avgOrderValue": { "$avg": "$payment.totalAmount" }
        }
    },

    # 2: Den Durchschnittswert in die nächste Pipelinephase übertragen
    {
        "$lookup": {
            "from": "orders",
            "pipeline": [
                {
                    "$match": {
                        # Filter: Bestellungen > 2 × Durchschnitt
                        "$expr": {
                            "$gt": [
                                "$payment.totalAmount",
                                { "$multiply": [ "$$avgValue", 2 ] }
                            ]
                        }
                    }
                },
                { "$group": { "_id": "$userId" } }
            ],
            "as": "usersAboveAverage",
            "let": { "avgValue": "$avgOrderValue" }
        }
    },

    # 3: Auflösen
    { "$unwind": "$usersAboveAverage" },

    # 4: Nutzerinformationen anhängen
    {
        "$lookup": {
            "from": "users",
            "localField": "usersAboveAverage._id",
            "foreignField": "_id",
            "as": "user"
        }
    },
    { "$unwind": "$user" },

    # 5: Ausgabe formatieren
    {
        "$project": {
            "_id": "$user._id",
            "name": "$user.name",
            "email": "$user.email"
        }
    }
]

result = list(db.orders.aggregate(pipeline))
result



## Task 1 – Count Orders per User
For each user, determine how many orders they have placed. Sort the results in descending order based on the order count.

**Expected output fields:**
- `userId`
- `name`
- `orderCount`

---

## Task 2 – Average Order Value
Compute the average total order value across all orders.
Additionally, include the minimum and maximum order value found.

## Task 3 – Product Revenue Ranking
For each product, calculate the total revenue generated.

**Definition:**
```
revenue = SUM(quantity × priceAtPurchase)
```

Sort products by total revenue in descending order.

**Expected output fields:**

- `productId`
- `name`
- `totalRevenue`

---

## Task 4 – User Classification by Spending Behavior
Classify each user based on their *average* order value.

**Categories:**
- `"low"`: < 100 EUR
- `"medium"`: 100–300 EUR
- `"high"`: > 300 EUR

**Expected output fields:**
- `_id`
- `name`
- `avgOrderValue`
- `category`

---

Attention - possible you have to adjust the category boundaries.


## Task 5 – Top 3 Users per Product Category by Spending
For each product category, determine the top 3 users ranked by total spending within that category.

**Rules:**
1. Join orders with products to determine the categories involved.
2. Compute the total spending per user per category
   (`quantity × priceAtPurchase` summed across all orders).
3. For each category, return the top three highest-spending users.

**Expected output structure example:**
```json
{
  "category": "electronics",
  "topUsers": [
    { "userId": "...", "name": "...", "totalSpent": 542.90 },
    { "userId": "...", "name": "...", "totalSpent": 301.50 },
    { "userId": "...", "name": "...", "totalSpent": 199.99 }
  ]
}
```
